## Keras ile IMDB dataset
Keras ile Imdb dataset' ini kullanarak yapılan binary classtification(ikili sınıflandırma) örneğidir. Imdb veri seti içinde `50000` adet filmin kritiğinin(sanırım film hakkında yapılan yorumlar) bulunduğu bir dataset. Film değerlendirmelerinden film hakkında olumlu veya olumsuz şekilde bir değerlendirme yapıyor. `Olumlu` ise `1` `olumsuz` ise `0`

Kaynak: Python ile Derin Öğrenme, Frainçois Chollet

Imdb veriseti keras' ta default olarak geliyor yani aslında Imdb veriseti için bir yapı oluşturulmuş yoksa veri hazır gelmiyor.

Veri seti aşağıdaki hücrede indiriliyor.

In [ ]:
from keras.datasets import imdb

(train_data, train_labes), (test_data, test_labels) = imdb.load_data(num_words=10000)

`load_data` Imdb verisetini indirip veriyi `list` dizileri şeklinde döndürüyor.

`num_words` ise listelerin içinde `[0, 100000)` aralığının dışında veri bulunmasını önlüyor.

In [ ]:
print("Imdb verisetinin ilk elemanının görünümü:\n", train_data[0])
print("\nİlk filmin kritiği: ", train_labes[0])

Yukarıdaki hücre çıktısında her sayı bir kelimeyi temsil etmektedir. `Kelime: Sayı` değerlerinin bulunduğu bir JSON dosyası bulunmaktadır.

In [ ]:
max([max(sequence) for sequence in train_data])
# Her film kritiğinde [0, 10000) aralığında kelime seçtiğimiz için bu işlemin neticesi 10000' nı geçemez.

In [ ]:
word_index = imdb.get_word_index()
print(f"\n\n'Kelime: Sayı' örneği: {list(word_index.keys())[0]}: {list(word_index.values())[0]}")

Yukarıdaki hücre ile kelimelerin sayı karşılığını tutan `JSON` dosyasını indirip onu bir `dict` haline getirdi: `Kelime: Sayı`

Aşağıdaki hücre ile `Kelime: Sayı` formatındaki `dict`' i `Sayı: Kelime` formatına gönüştürdü.

In [ ]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
print(f"'Sayı: Kelime' örneği: {list(reverse_word_index.keys())[0]}: {list(reverse_word_index.values())[0]}")

Aşağıdaki hücrede verileri hazırlıyoruz. Hazırlamaktan kasıt list olan verileri Numpy dizilerine çevirmek. Çünkü layer' lara list diziler vermek iyi bir fikir değil onun yerine tensor' lar göndermek iyi fikir.

Elimizdeki bir verileri iki şekilde tensor' a çevirebiliriz.
   1. Bütün verileri eşit boyuta getirip eksik kalan kısımları 0' lar ile doldurmak
   2. (verinin satır sayısı, kelime sayısı) şekilinde bir tensor oluşturup kelimelere karşılık gelen sayıların karışılığını 1 yapmak kalanlarını 0 yapmak. Mesela 560. kelime elimizde olsun. Oluşturduğumuz tensor' un gerekli satırının 560. index değerini 1 yapmak.

`vectorize_sequences` yukarıda bahsedilen ikinci yolu yapmaktadır.

`np.asarray().astype()` ile `label`' ları tutan dizileri numpy dizilerine çeviriyoruz ve `dtype` olarak `int` olan dizileri `float`' a çeviriyoruz.

In [ ]:
import numpy as np

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1
    return results


x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)
y_train = np.asanyarray(train_labes).astype('float32')
y_test = np.asanyarray(test_labels).astype('float32')

print(f"types: Data: {type(x_train)}, {type(y_train)}, Label: {type(x_test)}, {type(y_test)}")

In [ ]:
# sinir ağımızı oluşturturduk
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000, )))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
# Neural Network Compile
model.compile(
    optimizer='rmsprop',
    loss='binary_crossentropy',
    metrics=['acc']
)

In [ ]:
#Validation veriseti oluşturmak
x_val = x_train[:10000]
partial_x_train = x_train[10000:]

y_val = y_train[:10000]
partial_y_train = y_train[10000:]

In [ ]:
history = model.fit(
    partial_x_train,
    partial_y_train,
    epochs=20,
    batch_size=512,
    validation_data=(x_val, y_val)
)

In [ ]:
history_dict = history.history
history_dict.keys()